In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
zone_software = {1:{'V_Low':2,  'D_Low':0.2, 'DV_Low':0.4,   'V_Tr':2,	'D_Tr':0.5,	'DV_Tr':1},
                 2:{'V_Low':2,  'D_Low':0.1, 'DV_Low':0.2,   'V_Tr':2,	'D_Tr':0.3,	'DV_Tr':0.6},
                 3:{'V_Low':2,  'D_Low':0.2, 'DV_Low':0.4,   'V_Tr':2,	'D_Tr':0.5,	'DV_Tr':1},
                 4:{'V_Low':2,  'D_Low':0.3, 'DV_Low':0.6,   'V_Tr':2,	'D_Tr':1,	  'DV_Tr':2},
                 5:{'V_Low':1,  'D_Low':1,   'DV_Low':1,     'V_Tr':0,	'D_Tr':0,	  'DV_Tr':0},
                 6:{'V_Low':1,  'D_Low':1,   'DV_Low':1,     'V_Tr':0,	'D_Tr':0,	  'DV_Tr':0},
                 7:{'V_Low':2,  'D_Low':0.5, 'DV_Low':0.6,   'V_Tr':2,	'D_Tr':1.5,	'DV_Tr':3},
                 8:{'V_Low':0.5,'D_Low':0.05,'DV_Low':0.0025,'V_Tr':2,	'D_Tr':0.2,	'DV_Tr':0.4},
                 9:{'V_Low':0.5,'D_Low':0.05,'DV_Low':0.0025,'V_Tr':2,	'D_Tr':0.2,	'DV_Tr':0.4}}

# **Check the rasters times**  
Store only the rasters that are the same time velosity and depth

In [3]:
# create the paths for velocity and depth directories
v_path = r'data/Velocity/' # depth path
d_path = r'data/Depth/' # velocity path
d_tiffs = os.listdir(d_path)
v_tiffs = os.listdir(v_path)

In [4]:
dict_rasters_and_time = {}
# loop on rasters and check if the
for i in range(len(d_tiffs)):
    d_tiff = d_tiffs[i]
    v_tiff = d_tiff.replace('D','V')
    time = ''.join([s for s in v_tiff if s.isdigit()])
    if v_tiff in v_tiffs:
        dict_rasters_and_time[time] = (d_path+d_tiff,v_path+v_tiff)
        print('Both (', d_tiff,') and (',v_tiff,') exists')
    else:
        print(f'Time {time} is not found in both Velocity and Depth folders')

Time 0000 is not found in both Velocity and Depth folders
Time 0410 is not found in both Velocity and Depth folders
Time 0700 is not found in both Velocity and Depth folders
Both ( D0800.tif ) and ( V0800.tif ) exists
Both ( D1000.tif ) and ( V1000.tif ) exists
Both ( D1400.tif ) and ( V1400.tif ) exists
Time 1500 is not found in both Velocity and Depth folders


In [5]:
dict_rasters_and_time

{'0800': ('data/Depth/D0800.tif', 'data/Velocity/V0800.tif'),
 '1000': ('data/Depth/D1000.tif', 'data/Velocity/V1000.tif'),
 '1400': ('data/Depth/D1400.tif', 'data/Velocity/V1400.tif')}

# **Read the shapefile into GeoDataframe**

In [6]:
# gdf (GeoDataFrame)
shapefile_path = "data/Shp/Points.shp"
gdf = gpd.read_file(shapefile_path)

# Remove the null values of Zone_code

In [7]:
gdf = gdf[~gdf.Zone_Code1.isna()]

In [8]:
gdf = gdf.reset_index()

# **Adding V_Low, D_Low, DV_Low, V_Tr, D_Tr, DV_Tr**

In [9]:
# convert the Zone_code column into int value like in the dict_rasters_and_time above
gdf['Zone_Code1'] = gdf['Zone_Code1'].astype('int8')

In [10]:
zones = list(zone_software[1].keys())
# loop through the zone names and add each value 
for zone in zones:
    gdf[zone] = gdf['Zone_Code1'].apply(lambda x: zone_software[x][zone])

# **Function to retrive the pixel value from a raster in a specific long, lat**

In [11]:
from rasterstats import point_query

In [12]:
def pixel_value_from_coords(x, y, raster):
    """
    Function to retive the pixel value in a raster
    from a giving coordinates (lat, long)
    xy(tuple): (latitude,longitude) - 2 floats
    ratser(str): the raster path
    """
    point = {'type': 'Point', 'coordinates': (x,y)}
    return point_query(point, raster)[0]

# **Function to calculate R value**

In [13]:
def calculate_r_value(DV_Low , DV_Tr, V_low, D_low, V_Tr, D_Tr, V_i, D_i, Water=False):
    """
    This function calculates the value of R:
    a lot of condictions make the R value changed
    input:
        V_low[float]: Velocity low value from the table
        D_low[float]: Depth low value from the table
        V_Tr[float]: Velocity TR value from the table
        D_Tr[float]: Depth TR value from the table
        V_i[float]: Velocity value from the raster of specific time 
        D_i[float]: Depth low value from the raster of specific time
    """
    if Water :
        if D_i < D_low or V_i < V_low:
            R = 1
        else: #D_i >= D_Tr or V_i >= V_Tr: 
            R = 0
    else:
        if D_i < D_low or V_i < V_low:
            R = 1
        elif D_i > D_Tr or V_i > V_Tr:
            R = 0
        else:
            R = 1 - (DV_Tr-(D_i*V_i)/(DV_Tr-DV_Low))
            print(R)
    return R

In [14]:
dict_rasters_and_time

{'0800': ('data/Depth/D0800.tif', 'data/Velocity/V0800.tif'),
 '1000': ('data/Depth/D1000.tif', 'data/Velocity/V1000.tif'),
 '1400': ('data/Depth/D1400.tif', 'data/Velocity/V1400.tif')}

# **Adding the values of Velocity, Depth , (Velocity x Depth)**  
# **For each Time found**

In [15]:
def split_dataframe(df, chunk_size = 100):
    '''
    Function to split the dataframe into chunks
    '''
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size].index)
    return chunks

# Here you can choose the chunk size (in rows) 
# Its basically to split the dataframe into pieces to speed the job
# You can change it as you want

In [16]:
chunks = split_dataframe(gdf, chunk_size = 300)

In [17]:
# loop through chunks, and apply the function to the chunk
for chunk in chunks:
    for time in dict_rasters_and_time:
        rasters = dict_rasters_and_time[time]
        depth_raster = rasters[0]
        velocity_raster = rasters[1]
        gdf.loc[chunk, f'V_{time}'] = gdf.loc[chunk].apply(lambda x: (pixel_value_from_coords(x.x_coord,x.y_coord, velocity_raster)), axis=1)
        gdf.loc[chunk, f'D_{time}'] = gdf.loc[chunk].apply(lambda x: (pixel_value_from_coords(x.x_coord,x.y_coord,depth_raster)), axis=1)
    print(chunk)

RangeIndex(start=0, stop=300, step=1)
RangeIndex(start=300, stop=600, step=1)
RangeIndex(start=600, stop=900, step=1)
RangeIndex(start=900, stop=1200, step=1)
RangeIndex(start=1200, stop=1500, step=1)
RangeIndex(start=1500, stop=1800, step=1)
RangeIndex(start=1800, stop=2100, step=1)
RangeIndex(start=2100, stop=2400, step=1)
RangeIndex(start=2400, stop=2700, step=1)
RangeIndex(start=2700, stop=3000, step=1)
RangeIndex(start=3000, stop=3300, step=1)
RangeIndex(start=3300, stop=3600, step=1)
RangeIndex(start=3600, stop=3900, step=1)
RangeIndex(start=3900, stop=4200, step=1)
RangeIndex(start=4200, stop=4500, step=1)
RangeIndex(start=4500, stop=4800, step=1)
RangeIndex(start=4800, stop=5100, step=1)
RangeIndex(start=5100, stop=5400, step=1)
RangeIndex(start=5400, stop=5700, step=1)
RangeIndex(start=5700, stop=6000, step=1)
RangeIndex(start=6000, stop=6300, step=1)
RangeIndex(start=6300, stop=6600, step=1)
RangeIndex(start=6600, stop=6900, step=1)
RangeIndex(start=6900, stop=7200, step=1)
R

# **Replace the null Values with 0**

In [19]:
gdf = gdf.replace(np.nan, 0)

In [20]:
water_mask = gdf[gdf['Zone_Code1'].isin([5,6])]
water_mask_inverse = gdf[~gdf['Zone_Code1'].isin([5,6])]

# **Calculate R Value**

In [22]:
for time in dict_rasters_and_time:
    # Calculate the R value
    gdf.loc[water_mask.index, f'R_{time}'] = gdf.apply(lambda x: (calculate_r_value(x.DV_Low , x.DV_Tr, x.V_Low, x.D_Low, x.V_Tr, x.D_Tr, x[f'V_{time}'], x[f'D_{time}'], Water=True)), axis=1)
    gdf.loc[water_mask_inverse.index, f'R_{time}'] = gdf.apply(lambda x: (calculate_r_value(x.DV_Low , x.DV_Tr, x.V_Low, x.D_Low, x.V_Tr, x.D_Tr, x[f'V_{time}'], x[f'D_{time}'])), axis=1)
    print(f'R_{time} has been calculated')

R_0800 has been calculated
R_1000 has been calculated
R_1400 has been calculated


In [27]:
output_file_name = 'output.shp'
gdf.to_file(filename=output_file_name)

# **Export the shapefile as CSV**

In [25]:
csv_file_name = 'output.csv'
gdf.to_csv(csv_file_name, index=False)